In [21]:
# basics
import intake
import xarray as xr
import dask  # memory-efficient parallel computation and delayed execution (lazy evaluation).
import subprocess as sp

%run gem_helpers.ipynb

uid = getpass.getuser()
image_path = make_tempdir("intake_demo_plots")
data_cache_path = make_tempdir("intake_demo_data")

# this function is also part of the gem helpers
def get_list_from_cat(catalog, column):
    """A helper function for getting the contents of a column in an intake catalog.

    Call with the catalog to be inspected and the column of interest."""
    return sorted(catalog.unique(column)[column]["values"])

catalog_file = "/work/ka1081/Catalogs/dyamond-nextgems.json"
cat = intake.open_esm_datastore(catalog_file)


In [22]:
fre='1hour'
var='Qbot'
hits = cat.search(simulation_id=["ngc2009", "ngc2012", "ngc2013"], frequency=fre, variable_id=var)

In [23]:
get_from_cat(hits,["grid_label","grid_id","simulation_id", "variable_id","frequency"])

,grid_label,grid_id,simulation_id,variable_id,frequency
0,gn,375cb0cc-637e-11e8-9d6f-8f41a9b9ff4b,ngc2009,"(atmos_fluxes_FrshFlux_Evaporation, atmos_fluxes_FrshFlux_Precipitation, atmos_fluxes_FrshFlux_Runoff, atmos_fluxes_FrshFlux_SnowFall, atmos_fluxes_HeatFlux_Latent, atmos_fluxes_HeatFlux_LongWave, atmos_fluxes_HeatFlux_Sensible, atmos_fluxes_HeatFlux_ShortWave, atmos_fluxes_HeatFlux_Total, atmos_fluxes_stress_x, atmos_fluxes_stress_xw, atmos_fluxes_stress_y, atmos_fluxes_stress_yw, Qbot, Qtop)",1hour
1,gn,66c2eb2c-9bd9-11e8-97bc-e1d6091d8653,ngc2013,"(atmos_fluxes_FrshFlux_Evaporation, atmos_fluxes_FrshFlux_Precipitation, atmos_fluxes_FrshFlux_Runoff, atmos_fluxes_FrshFlux_SnowFall, atmos_fluxes_HeatFlux_Latent, atmos_fluxes_HeatFlux_Sensible, atmos_fluxes_HeatFlux_LongWave, atmos_fluxes_HeatFlux_ShortWave, atmos_fluxes_HeatFlux_Total, atmos_fluxes_stress_x, atmos_fluxes_stress_y, atmos_fluxes_stress_xw, atmos_fluxes_stress_yw, Qtop, Qbot)",1hour
2,gn,not implemented,ngc2012,"(atmos_fluxes_FrshFlux_Evaporation, atmos_fluxes_FrshFlux_Precipitation, atmos_fluxes_FrshFlux_Runoff, atmos_fluxes_FrshFlux_SnowFall, atmos_fluxes_HeatFlux_Latent, atmos_fluxes_HeatFlux_Sensible, atmos_fluxes_HeatFlux_LongWave, atmos_fluxes_HeatFlux_ShortWave, atmos_fluxes_HeatFlux_Total, atmos_fluxes_stress_x, atmos_fluxes_stress_y, atmos_fluxes_stress_xw, atmos_fluxes_stress_yw, Qtop, Qbot)",1hour


In [26]:

file_cat = {}
for simulation_id in ("ngc2012", "ngc2013"):
#for simulation_id in ("ngc2009"):
    file_cat[simulation_id] = get_list_from_cat(
        hits.search(simulation_id=simulation_id), "uri"
    )

In [27]:


outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
#                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '4', '-daymean', '-sellonlatbox,300,310,5,17', '-cat', '-apply,-selname,Qbot', '[', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200120T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200121T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200122T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200123T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200124T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200125T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_oce_2d_1h_mean_20200126T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2012

In [26]:
for files in file_cat:
    fileS=files.split(',')
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True:
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + fileS[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '4', '-remapcon,r360x180', '-daymean', '-sellonlatbox,300,310,5,17', '-cat', '-apply,-selname,sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_atm_2d_3h_mean_20200120T000000Z.nc', ']', '/scratch/b/b381639/intake_demo_data/sfcwind_daymean_ngc2012.nc']
cdo(1) daymean: Process started
cdo(2) sellonlatbox: Process started
cdo(3) cat: Process started
cdo(4) selname: Process started
cdo    remapcon: YAC first order conservative weights from unstructured (14701) to lonlat (360x180) grid
cdo(4) selname: Processed 503316480 values from 26 variables over 96 timesteps.
cdo(3) cat: Processed 503316480 values from 1 variable over 96 timesteps.
cdo(2) sellonlatbox: Processed 503316480 values from 1 variable over 96 timesteps.
cdo(1) daymean: Processed 1411296 values from 1 variable over 107 timesteps.
cdo    remapcon: Processed 176412 values from 1 variable over 12 timesteps [10.88s 2043MB].


NameError: name 'outfile_dict' is not defined

In [71]:
outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

'/scratch/b/b381639/intake_demo_data/sfcwind_daymean_1.nc'

In [19]:
file_cat = {}
#for simulation_id in ("ngc2009", "ngc2012"):
for simulation_id in ("ngc2009","ngc2012"):
    file_cat[simulation_id] = get_list_from_cat(
        hits.search(simulation_id=simulation_id), "uri"
    )

outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:106: UserWarning: Query returned zero results.
  warn(message)
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = df[columns].apply(_find_unique, result_type='reduce').to_dict()


ValueError: Length of values (0) does not match length of index (1)